In [2]:
pip install scanpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82808 sha256=21635625b6fcfaf28ae167a6ec95b6964c92758fd266c066437829f388eb8222
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8027 sha256=47e19df3d37179f75f4c3b07d3bafd47bffd13b1745c38be4822a9dcfe6e063a
  Stored in directory: /root/.cache/pip/wheels/6a/

In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as an

In [4]:
import os

In [5]:
dir_path="/content/drive/MyDrive/temp_project/shrew/QC/"

In [6]:
adata=sc.read_h5ad(dir_path+"treeshrew_filt.h5ad")
adata

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 1948 × 15471
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_pct_counts_hb_colors', 'qc_pct_counts_mt_colors', 'qc_pct_counts_ribo_colors', 'qc_total_counts_colors', 'spatial'
    obsm: 'spatial'

In [8]:
adata.obs.cluster

AAACAAGTATCTCCCA-1    L5
AAACAGGGTCTATATT-1    L3
AAACCCGAACGAAATC-1    L1
AAACCGGAAATGTTAA-1    L5
AAACTTGCAAACGTAT-1    L3
                      ..
TTGTCACCGCGGTATC-1    L3
TTGTGAACCTAATCCG-1    L6
TTGTGGTAGGAGGGAT-1    L5
TTGTGGTGGTACTAAG-1    WM
TTGTTTCATTAGTCTA-1    WM
Name: cluster, Length: 1948, dtype: category
Categories (7, object): ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'WM']

In [7]:
adata.var

,gene_ids,feature_types,genome,mt,hb,ribo
DMD,ENSTBEG00000005582,Gene Expression,Tupaia_belangeri_genome,False,False,False
WNT5A,ENSTBEG00000016004,Gene Expression,Tupaia_belangeri_genome,False,False,False
ERC2,ENSTBEG00000016216,Gene Expression,Tupaia_belangeri_genome,False,False,False
CCDC66,ENSTBEG00000016513,Gene Expression,Tupaia_belangeri_genome,False,False,False
TASOR,ENSTBEG00000016784,Gene Expression,Tupaia_belangeri_genome,False,False,False
...,...,...,...,...,...,...
PSKH1,ENSTBEG00000010830,Gene Expression,Tupaia_belangeri_genome,False,False,False
ENSTBEG00000016427,ENSTBEG00000016427,Gene Expression,Tupaia_belangeri_genome,False,False,False
ENSTBEG00000014868,ENSTBEG00000014868,Gene Expression,Tupaia_belangeri_genome,False,False,False
ENSTBEG00000008850,ENSTBEG00000008850,Gene Expression,Tupaia_belangeri_genome,False,False,False


In [9]:
# add pseudobulk label
adata.obs['pb_label'] = adata.obs.cluster.astype(str) + '.treeshrew_' + adata.obs.sample_id.astype(str)
# create pseudobulk df
sample_layer_list = adata.obs.pb_label.unique().tolist()
# create pseudobulk sum df
pb_sum_list = [adata[adata.obs.pb_label == sample].X.sum(axis=0) for sample in sample_layer_list]
pb_sum_df = pd.DataFrame(np.concatenate(pb_sum_list).T, columns=sample_layer_list, index=adata.var_names)
# create pseudobulk mean df
pb_mean_list = [adata[adata.obs.pb_label == sample].X.mean(axis=0) for sample in sample_layer_list]
pb_mean_df = pd.DataFrame(np.concatenate(pb_mean_list).T, columns=sample_layer_list, index=adata.var_names)
# create annotation file
columns = ['layer', 'sample_id']
annotation_list = [sample.split('.') for sample in sample_layer_list]
annotation = pd.DataFrame(annotation_list, index=sample_layer_list, columns=columns)
annotation['condition'] = "treeshrew"
#save files
pb_sum_df.to_csv(f'/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/expression_sum_treeshrew.csv')
pb_mean_df.to_csv(f'/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/expression_mean_treeshrew.csv')
annotation.to_csv(f'/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/annotation_treeshrew.csv')

In [10]:
dir_path="/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/"

In [11]:
exp_sum=pd.read_csv(dir_path+"expression_sum_treeshrew.csv",index_col=0)
exp_mean=pd.read_csv(dir_path+"expression_mean_treeshrew.csv",index_col=0)
ann=pd.read_csv(dir_path+"annotation_treeshrew.csv",index_col=0)

In [12]:
adata_sum = an.AnnData(exp_sum.T)
adata_sum.obs = ann
adata_sum

AnnData object with n_obs × n_vars = 28 × 15471
    obs: 'layer', 'sample_id', 'condition'

In [13]:
adata_sum.obs

,layer,sample_id,condition
L5.treeshrew_s31,L5,treeshrew_s31,treeshrew
L3.treeshrew_s31,L3,treeshrew_s31,treeshrew
L1.treeshrew_s31,L1,treeshrew_s31,treeshrew
L6.treeshrew_s31,L6,treeshrew_s31,treeshrew
WM.treeshrew_s31,WM,treeshrew_s31,treeshrew
L4.treeshrew_s31,L4,treeshrew_s31,treeshrew
L2.treeshrew_s31,L2,treeshrew_s31,treeshrew
L6.treeshrew_s32,L6,treeshrew_s32,treeshrew
L5.treeshrew_s32,L5,treeshrew_s32,treeshrew
L1.treeshrew_s32,L1,treeshrew_s32,treeshrew


In [14]:
adata_mean = an.AnnData(exp_mean.T)
adata_mean.obs = ann
adata_mean

AnnData object with n_obs × n_vars = 28 × 15471
    obs: 'layer', 'sample_id', 'condition'

In [15]:
adata_mean.obs

,layer,sample_id,condition
L5.treeshrew_s31,L5,treeshrew_s31,treeshrew
L3.treeshrew_s31,L3,treeshrew_s31,treeshrew
L1.treeshrew_s31,L1,treeshrew_s31,treeshrew
L6.treeshrew_s31,L6,treeshrew_s31,treeshrew
WM.treeshrew_s31,WM,treeshrew_s31,treeshrew
L4.treeshrew_s31,L4,treeshrew_s31,treeshrew
L2.treeshrew_s31,L2,treeshrew_s31,treeshrew
L6.treeshrew_s32,L6,treeshrew_s32,treeshrew
L5.treeshrew_s32,L5,treeshrew_s32,treeshrew
L1.treeshrew_s32,L1,treeshrew_s32,treeshrew


In [47]:
adata_mean.write_h5ad("/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/ts_pb_mean.h5ad")
adata_sum.write_h5ad("/content/drive/MyDrive/temp_project/shrew/pseudobulk layer/ts_pb_sum.h5ad")

In [16]:
hexp_mean=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/expression_mean_human.csv",index_col=0)
hann=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/annotation_human.csv",index_col=0)
hadata_mean = an.AnnData(hexp_mean.T)
hadata_mean.obs = hann
hadata_mean

AnnData object with n_obs × n_vars = 35 × 19966
    obs: 'layer', 'sample_id', 'condition'

In [20]:
cexp_mean=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/expression_mean_chimp.csv",index_col=0)
cann=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/annotation_chimp.csv",index_col=0)
cadata_mean = an.AnnData(cexp_mean.T)
cadata_mean.obs = cann
cadata_mean_filt=cadata_mean[cadata_mean.obs.sample_id != 'chimp_13302']
cadata_mean_filt

View of AnnData object with n_obs × n_vars = 26 × 25080
    obs: 'layer', 'sample_id', 'condition'

In [21]:
mexp_mean=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/expression_mean_macaque.csv",index_col=0)
mann=pd.read_csv("/content/drive/MyDrive/temp_project/Spatial project/data/pseudobulks/annotation_macaque.csv",index_col=0)
madata_mean = an.AnnData(mexp_mean.T)
madata_mean.obs = mann
madata_mean

AnnData object with n_obs × n_vars = 28 × 21591
    obs: 'layer', 'sample_id', 'condition'

In [22]:
spe=[adata_mean,hadata_mean,cadata_mean,madata_mean]
evo=an.concat(spe, merge='same')

In [23]:
evo

AnnData object with n_obs × n_vars = 122 × 8991
    obs: 'layer', 'sample_id', 'condition'

In [24]:
evo.write_h5ad("four_spe_pseudobulk_filt.h5ad")